# Diseño del Modelo - PASCAL VOC 2007 Multilabel

## Objetivo
Diseñar arquitectura para clasificacion multilabel usando PASCAL VOC 2007.

Dataset: 20 categorias de objetos con multilabel REAL
Modelo: EfficientNetB0 + Binary Cross-Entropy + Sigmoid

---
## 1. Importación de Librerías

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# TensorFlow / Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import (
    EfficientNetB0, 
    ResNet50V2, 
    MobileNetV2
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Métricas
from sklearn.metrics import (
    hamming_loss, 
    f1_score, 
    precision_score, 
    recall_score,
    accuracy_score
)

print(f" TensorFlow version: {tf.__version__}")
print(f" GPU disponible: {tf.config.list_physical_devices('GPU')}")

# Configuración para reproducibilidad
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

 TensorFlow version: 2.20.0
 GPU disponible: []


---
## 2. Configuración del Proyecto

In [ ]:
# Configuracion
PROJECT_ROOT = Path(os.getcwd()).parent
DATA_DIR = PROJECT_ROOT / 'data' / 'voc2007'
MODELS_DIR = PROJECT_ROOT / 'models'
MODELS_DIR.mkdir(exist_ok=True)

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.0005

print(f"Configuracion para PASCAL VOC 2007")
print(f"Datos: {DATA_DIR}")
print(f"Modelos: {MODELS_DIR}")

Configuracion para MS COCO Food
Datos: c:\Users\mlata\Documents\iajordy2\data\food101
Modelos: c:\Users\mlata\Documents\iajordy2\models


---
## 3. Cargar Datos del Análisis Previo

In [3]:
# Cargar clases
with open(DATA_DIR / 'classes.json', 'r') as f:
    classes = json.load(f)
NUM_CLASSES = len(classes)
print(f"Clases: {NUM_CLASSES}")
print(f"Ejemplos: {classes[:5]}")

Clases: 10
Ejemplos: ['banana', 'apple', 'sandwich', 'orange', 'broccoli']


---
## 4. Definición Formal del Problema Multilabel

###  Problema:
Dado un conjunto de imágenes de alimentos $X = \{x_1, x_2, ..., x_n\}$ y un conjunto de etiquetas $L = \{l_1, l_2, ..., l_m\}$, queremos aprender una función:

$$f: X \rightarrow 2^L$$

Donde $2^L$ es el conjunto potencia de $L$, es decir, cada imagen puede tener un subconjunto de etiquetas.

###  Formulación Matemática:

Para cada imagen $x_i$, el modelo produce un vector de probabilidades:

$$\hat{y}_i = [p_1, p_2, ..., p_m]$$

Donde cada $p_j \in [0, 1]$ representa la probabilidad de que la clase $j$ esté presente en la imagen.

###  Diferencias con Multiclase:

| Aspecto | Multiclase | Multilabel |
|---------|-----------|------------|
| **Activación** | Softmax: $\sigma(z)_j = \frac{e^{z_j}}{\sum_{k=1}^{m} e^{z_k}}$ | Sigmoid: $\sigma(z)_j = \frac{1}{1 + e^{-z_j}}$ |
| **Loss** | Categorical Cross-Entropy | Binary Cross-Entropy |
| **Output** | $\sum p_j = 1$ | Cada $p_j$ independiente |
| **Threshold** | argmax | Threshold (ej. 0.5) |

###  Binary Cross-Entropy Loss:

$$L = -\frac{1}{N} \sum_{i=1}^{N} \sum_{j=1}^{m} \left[ y_{ij} \log(\hat{y}_{ij}) + (1-y_{ij}) \log(1-\hat{y}_{ij}) \right]$$

Donde:
- $N$ = número de muestras
- $m$ = número de clases
- $y_{ij}$ = etiqueta verdadera (0 o 1)
- $\hat{y}_{ij}$ = predicción del modelo

---
## 5. Métricas de Evaluación Multilabel

### 1️ Hamming Loss
Mide la fracción de etiquetas incorrectamente predichas:

$$\text{Hamming Loss} = \frac{1}{N \cdot m} \sum_{i=1}^{N} \sum_{j=1}^{m} \mathbb{1}(y_{ij} \neq \hat{y}_{ij})$$

**Interpretación**: Menor es mejor (0 = perfecto)

### 2️ F1-Score (Macro/Micro)

**Macro F1**: Promedio de F1 por clase
$$F1_{\text{macro}} = \frac{1}{m} \sum_{j=1}^{m} F1_j$$

**Micro F1**: F1 global considerando todos los pares
$$F1_{\text{micro}} = \frac{2 \cdot TP}{2 \cdot TP + FP + FN}$$

### 3️ Precision y Recall

$$\text{Precision} = \frac{TP}{TP + FP}$$

$$\text{Recall} = \frac{TP}{TP + FN}$$

### 4️ Subset Accuracy
Porcentaje de predicciones donde **todas** las etiquetas coinciden exactamente:

$$\text{Accuracy}_{\text{subset}} = \frac{1}{N} \sum_{i=1}^{N} \mathbb{1}(y_i = \hat{y}_i)$$

---
## 6. Arquitectura del Modelo: Transfer Learning

###  Arquitectura Propuesta:

```
INPUT (224x224x3)
    ↓
BACKBONE: EfficientNetB0 (Pre-trained on ImageNet)
    ↓
GlobalAveragePooling2D
    ↓
Dense(512, activation='relu')
    ↓
Dropout(0.5)
    ↓
Dense(256, activation='relu')
    ↓
Dropout(0.3)
    ↓
Dense(NUM_CLASSES, activation='sigmoid')  ← MULTILABEL
    ↓
OUTPUT (probabilities per class)
```

###  Justificación:

1. **EfficientNetB0**:
   - Balance entre precisión y eficiencia
   - Pre-entrenado en ImageNet (conocimiento transferido)
   - Solo 5.3M parámetros (más ligero que ResNet50)

2. **GlobalAveragePooling2D**:
   - Reduce dimensionalidad espacial
   - Menos propenso a overfitting que Flatten

3. **Capas Dense + Dropout**:
   - Adaptación al dominio específico (comida)
   - Dropout para regularización

4. **Activación Sigmoid**:
   - CRÍTICO para multilabel
   - Cada neurona produce probabilidad independiente
   - Permite múltiples clases activas simultáneamente

In [4]:
# Modelo multilabel óptimo con EfficientNet y configuración correcta
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models, optimizers
def create_multilabel_model(num_classes, img_size=(224, 224)):
    inputs = layers.Input(shape=(*img_size, 3))
    base_model = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=inputs, pooling=None)
    base_model.trainable = False
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    return model, base_model

# Crear modelo
model, base_model = create_multilabel_model(num_classes=NUM_CLASSES, img_size=IMG_SIZE)
print("Modelo multilabel creado correctamente")
model.summary()

Modelo multilabel creado correctamente


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,839,341 (18.46 MB)

 Trainable params: 789,770 (3.01 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

---
## 7. Compilación del Modelo

In [5]:
# Compilación óptima para multilabel
loss_function = 'binary_crossentropy'
optimizer = optimizers.Adam(learning_rate=LEARNING_RATE)
metrics = [
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc', multi_label=True)
    # Puedes agregar F1-score y Hamming Loss en validación manual
 ]
model.compile(
    optimizer=optimizer,
    loss=loss_function,
    metrics=metrics
)
print("Modelo compilado para multilabel")

Modelo compilado para multilabel


---
## 8. Justificación de Decisiones Arquitectónicas

###  1. ¿Por qué Binary Cross-Entropy?

**Comparación:**

| Loss | Uso | Fórmula | Problema |
|------|-----|---------|----------|
| **Categorical CE** | Multiclase (exclusiva) | $-\sum y_i \log(\hat{y}_i)$ | Asume una sola clase |
| **Binary CE** | Multilabel | $-\sum [y_i \log(\hat{y}_i) + (1-y_i)\log(1-\hat{y}_i)]$ | Trata cada clase independientemente |

**Ejemplo:**
- Imagen con: `[rice, fish, vegetables]`
- Target: `[0,0,1,0,1,0,0,1,0,...]` (vector binario)
- Binary CE evalúa cada posición independientemente
- Categorical CE fallaría porque espera un solo 1

###  2. ¿Por qué Sigmoid en lugar de Softmax?

**Softmax (INCORRECTO para multilabel):**
$$p_i = \frac{e^{z_i}}{\sum_{j} e^{z_j}} \quad \rightarrow \quad \sum p_i = 1$$

- Fuerza competencia entre clases
- Solo una clase puede dominar
- Probabilidades suman 1

**Sigmoid (CORRECTO para multilabel):**
$$p_i = \frac{1}{1 + e^{-z_i}}$$

- Cada clase es independiente
- Múltiples clases pueden tener alta probabilidad
- Probabilidades NO suman 1

**Ejemplo práctico:**
```python
# Softmax output (MALO para multilabel)
[0.05, 0.70, 0.03, 0.15, 0.07]  # Solo 'fish' domina

# Sigmoid output (BUENO para multilabel)
[0.82, 0.91, 0.15, 0.88, 0.23]  # 'rice', 'fish', 'vegetables' activos
```

###  3. ¿Por qué Transfer Learning?

**Ventajas:**
1. **Convergencia más rápida**: Pesos pre-entrenados
2. **Mejor generalización**: Características aprendidas de ImageNet
3. **Menos datos necesarios**: Aprovecha conocimiento previo
4. **Menor overfitting**: Regularización implícita

**Estrategia:**
- Fase 1: Entrenar solo clasificador (backbone congelado)
- Fase 2: Fine-tuning (descongelar últimas capas del backbone)

---
## 9. Visualización de la Arquitectura

In [6]:
# Contar parámetros
total_params = model.count_params()
trainable_params = sum([tf.size(w).numpy() for w in model.trainable_weights])
non_trainable_params = total_params - trainable_params

print(" ESTADÍSTICAS DEL MODELO\n")
print("="*60)
print(f"Total de parámetros:      {total_params:,}")
print(f"Parámetros entrenables:   {trainable_params:,}")
print(f"Parámetros congelados:    {non_trainable_params:,}")
print("="*60)

print("\n Capas del modelo:")
for i, layer in enumerate(model.layers[-10:], 1):  # Últimas 10 capas
    trainable = "✓" if layer.trainable else "✗"
    print(f"  {trainable} {layer.name}: {layer.__class__.__name__}")

print(f"\n Backbone: {base_model.name}")
print(f"   Capas en backbone: {len(base_model.layers)}")
print(f"   Congelado: {'Sí' if not base_model.trainable else 'No'}")

 ESTADÍSTICAS DEL MODELO

Total de parámetros:      4,839,341
Parámetros entrenables:   789,770
Parámetros congelados:    4,049,571

 Capas del modelo:
  ✗ block7a_project_bn: BatchNormalization
  ✗ top_conv: Conv2D
  ✗ top_bn: BatchNormalization
  ✗ top_activation: Activation
  ✓ global_average_pooling2d: GlobalAveragePooling2D
  ✓ dense: Dense
  ✓ dropout: Dropout
  ✓ dense_1: Dense
  ✓ dropout_1: Dropout
  ✓ dense_2: Dense

 Backbone: efficientnetb0
   Capas en backbone: 238
   Congelado: Sí


---
## 10. Función de Threshold para Predicciones

In [7]:
def apply_threshold(predictions, threshold=0.5):
    return (predictions >= threshold).astype(int)
print("Función de threshold definida para multilabel")

Función de threshold definida para multilabel


---
## 11. Métricas Personalizadas para Multilabel

In [8]:
from sklearn.metrics import hamming_loss, f1_score, precision_score, recall_score, accuracy_score
def compute_multilabel_metrics(y_true, y_pred, threshold=0.5):
    y_pred_binary = (y_pred >= threshold).astype(int)
    metrics = {
        'hamming_loss': hamming_loss(y_true, y_pred_binary),
        'subset_accuracy': accuracy_score(y_true, y_pred_binary),
        'f1_micro': f1_score(y_true, y_pred_binary, average='micro'),
        'f1_macro': f1_score(y_true, y_pred_binary, average='macro'),
        'precision_micro': precision_score(y_true, y_pred_binary, average='micro'),
        'recall_micro': recall_score(y_true, y_pred_binary, average='micro'),
    }
    return metrics
print("Función de métricas multilabel definida")

Función de métricas multilabel definida


---
## 12. Callbacks para Entrenamiento

In [9]:
# Callbacks recomendados para entrenamiento robusto
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1),
    keras.callbacks.ModelCheckpoint(filepath=str(MODELS_DIR / 'best_model.h5'), monitor='val_loss', save_best_only=True, verbose=1),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, verbose=1)
    # Puedes agregar TensorBoard si lo necesitas
 ]
print("Callbacks configurados para entrenamiento multilabel")

Callbacks configurados para entrenamiento multilabel


---
## 13. Guardar Configuración del Modelo

In [10]:
# Configuración completa
model_config = {
    'architecture': {
        'backbone': 'EfficientNetB0',
        'input_shape': list(IMG_SIZE) + [3],
        'num_classes': NUM_CLASSES,
        'classifier_layers': [512, 256],
        'dropout_rates': [0.5, 0.3],
        'output_activation': 'sigmoid'
    },
    'training': {
        'loss': 'binary_crossentropy',
        'optimizer': 'Adam',
        'learning_rate': LEARNING_RATE,
        'batch_size': BATCH_SIZE,
        'epochs': EPOCHS,
    },
    'metrics': [
        'accuracy', 'precision', 'recall', 'auc',
        'hamming_loss', 'f1_micro', 'f1_macro'
    ],
    'classes': classes
}

# Guardar configuración
with open(MODELS_DIR / 'model_config.json', 'w') as f:
    json.dump(model_config, f, indent=2)

print(" Configuración guardada en:", MODELS_DIR / 'model_config.json')

 Configuración guardada en: c:\Users\mlata\Documents\iajordy2\models\model_config.json


---
## 14. Resumen y Conclusiones

###  Modelo Diseñado:
- **Arquitectura**: EfficientNetB0 + Custom Classifier
- **Tipo**: Clasificación Multilabel
- **Activación**: Sigmoid (crítico para multilabel)
- **Loss**: Binary Cross-Entropy
- **Clases**: 30+ categorías de alimentos

###  Decisiones Clave:
1. **Sigmoid vs Softmax**: Permite múltiples etiquetas simultáneas
2. **Binary CE vs Categorical CE**: Trata cada clase independientemente
3. **Transfer Learning**: Aprovecha conocimiento de ImageNet
4. **Dropout**: Regularización para evitar overfitting

###  Métricas a Monitorear:
- **Hamming Loss**: Fracción de etiquetas incorrectas
- **F1-Score**: Balance entre precision y recall
- **Subset Accuracy**: Predicciones completamente correctas

###  Siguiente Paso:
Ir al notebook **03_training_retraining.ipynb** para entrenar y hacer fine-tuning del modelo.